## 2.3 **깁스 샘플링을 활용한 구조변화 모형 추정**

### 2.3.1. 모델 설정

2.3.3은 구조변화 시점 $\tau$를 추정하는 **베이지안 구조변화 모형**입니다. 주요 개념은 $\tau$ 이전과 이후의 데이터가 각각 다른 회귀 계수 $\beta_1$과 $\beta_2$를 가진다고 가정하는 것입니다.

#### 기본 설정
1. **구조변화 시점** $ \tau $:  $ \tau $는 구조변화 시점으로, 데이터의 시간 축에서 어느 시점에 변화가 일어났는지를 나타냅니다.
   
2. **회귀 계수** $ \beta_1, \beta_2 $: 
   - $ \beta_1 $은 구조변화 이전 구간의 회귀 계수.
   - $ \beta_2 $는 구조변화 이후 구간의 회귀 계수.

3. **분산** $ \sigma_1^2, \sigma_2^2 $: 
   - $ \sigma_1^2 $는 구조변화 이전 구간의 오차 분산.
   - $ \sigma_2^2 $는 구조변화 이후 구간의 오차 분산.

#### 모델 정의:
- 구조변화 이전 ($ t \leq \tau $) 데이터에 대한 관측 모델:
  $
  y_t | \beta_1, \sigma_1^2 \sim \mathcal{N}(X_t \beta_1, \sigma_1^2) \quad \text{for} \, t \leq \tau
  $
  
- 구조변화 이후 ($ t > \tau $) 데이터에 대한 관측 모델:
  $
  y_t | \beta_2, \sigma_2^2 \sim \mathcal{N}(X_t \beta_2, \sigma_2^2) \quad \text{for} \, t > \tau
  $

여기서, $ X_t $는 설명변수 행렬, $ y_t $는 종속변수(여기서는 물가상승률)입니다.


### 2.3.2. 사후 분포 샘플링

베이지안 방법론을 사용하기 때문에, 사전분포(prior)와 사후분포(posterior)를 기반으로 파라미터를 추정합니다.

1. **회귀 계수의 사전분포**:
   $
   \beta_j \sim \mathcal{N}(\beta_0, B_0) \quad \text{for} \, j = 1, 2
   $
   
   여기서 $ \beta_0 $는 사전분포의 평균, $ B_0 $는 공분산 행렬입니다.


2. **오차 분산의 사전분포**:
   $
   \sigma_j^2 \sim \text{InverseGamma}\left( \frac{\alpha_0}{2}, \frac{\delta_0}{2} \right) \quad \text{for} \, j = 1, 2
   $
   
   여기서 $ \alpha_0 $와 $ \delta_0 $는 주어진 하이퍼파라미터입니다.


3. **구조변화 시점 $ \tau $의 사전분포**:
   $
   \pi(\tau) = \frac{1}{T}, \quad \tau \in \{1, 2, ..., T-1\}
   $
   $ \tau $는 **균등분포(Uniform distribution)** 를 따르며, 구조변화 시점이 어느 시점에서든 발생할 가능성이 동일하다고 가정합니다.

Gibbs 샘플링 절차

1. **파라미터 업데이트**:
   - 각 파라미터를 **완전 조건부 분포**에 따라 순차적으로 샘플링합니다.

2. **구조변화 시점 $\tau$의 조건부 분포**:
   $
   \pi(\tau | \beta_1, \beta_2, \sigma_1^2, \sigma_2^2, y) \propto \prod_{t=1}^{\tau} \mathcal{N}(y_t | X_t \beta_1, \sigma_1^2) \times \prod_{t=\tau+1}^{T} \mathcal{N}(y_t | X_t \beta_2, \sigma_2^2)
   $
   
   $\tau$를 추정하는 데 사용되는 조건부 분포는 $\beta_1, \beta_2$, 그리고 각 구간에서의 오차 분산 $\sigma_1^2, \sigma_2^2$와 관측값 $ y $에 의존합니다.


3. **구조변화 이전 및 이후 파라미터의 조건부 분포**:
   - **$\beta_1$** 와 **$\sigma_1^2$** 은 $ t \leq \tau $ 구간에서 데이터에 의해 업데이트됩니다.
   - **$\beta_2$** 와 **$\sigma_2^2$** 은 $ t > \tau $ 구간에서 데이터에 의해 업데이트됩니다.


4. **오차 분산의 완전 조건부 분포**:
   $
   \sigma_j^2 | \text{other parameters} \sim \text{InverseGamma}\left( \frac{\alpha_0 + n_j}{2}, \frac{\delta_0 + \text{SSR}_j}{2} \right)
   $
   여기서 $ n_j $는 해당 구간의 데이터 수, $ \text{SSR}_j $는 해당 구간에서의 잔차 제곱합입니다.

이 그림은 **알고리즘 2.2: 구조변화모형 깁스 샘플링**에 대한 내용을 요약한 것으로, 구조변화 시점을 추정하는 베이지안 모형을 깁스 샘플링을 통해 추정하는 절차를 단계별로 설명하고 있습니다. 각 단계는 $\tau$ (구조변화 시점), 회귀계수 $\beta_1, \beta_2$, 분산 $\sigma_1^2, \sigma_2^2$을 차례로 샘플링하는 방법을 기술합니다.

### 각 단계 설명

#### **0단계**: 초기값 설정
- 초기값 $\sigma_1^{2(0)}, \sigma_2^{2(0)}, \tau^{(0)}$를 사전 정보나 임의의 평균값으로 설정합니다.
- $j$를 1로 시작하여 샘플링을 반복할 준비를 합니다.

#### **1단계**: 데이터 나누기
- 이전 단계에서 추정된 $\tau^{(j-1)}$를 바탕으로 데이터를 구조변화 이전 ($Y_1, X_1$)과 이후 ($Y_2, X_2$)로 나눕니다.
  - $Y_1, X_1$: 구조변화 이전 구간의 종속변수 및 설명변수.
  - $Y_2, X_2$: 구조변화 이후 구간의 종속변수 및 설명변수.

#### **2단계**: $\beta_1$ 샘플링
- **완전 조건부 분포**에서 구조변화 이전 구간에 대한 회귀계수 $\beta_1$을 샘플링합니다.
- $
\beta_1 | Y_1, \sigma_1^{2(j-1)} \sim \mathcal{N}(\mu_{\beta_1}, \Sigma_{\beta_1})
$
  - $\beta_1$는 $Y_1$과 $\sigma_1^2$에 따라 샘플링됩니다.

#### **3단계**: $\sigma_1^2$ 샘플링
- 구조변화 이전 구간의 오차 분산 $\sigma_1^2$를 샘플링합니다.
- $
\sigma_1^2 | Y_1, \beta_1^{(j)} \sim \text{InverseGamma}(\alpha_1, \beta_1)
$
  - $\sigma_1^2$는 $Y_1$과 $\beta_1^{(j)}$에 의존하여 업데이트됩니다.

#### **4단계**: $\beta_2$ 샘플링
- 구조변화 이후 구간에 대한 회귀계수 $\beta_2$를 샘플링합니다.
- $
\beta_2 | Y_2, \sigma_2^{2(j-1)} \sim \mathcal{N}(\mu_{\beta_2}, \Sigma_{\beta_2})
$
  - $\beta_2$는 $Y_2$와 $\sigma_2^2$에 의존하여 샘플링됩니다.

#### **5단계**: $\sigma_2^2$ 샘플링
- 구조변화 이후 구간의 오차 분산 $\sigma_2^2$를 샘플링합니다.
- $
\sigma_2^2 | Y_2, \beta_2^{(j)} \sim \text{InverseGamma}(\alpha_2, \beta_2)
$
  - $\sigma_2^2$는 $Y_2$와 $\beta_2^{(j)}$에 따라 샘플링됩니다.

#### **6단계**: $\tau$ 샘플링
- 구조변화 시점 $\tau$를 완전 조건부 분포에서 샘플링합니다.
- $
\tau | Y, \beta_1^{(j)}, \beta_2^{(j)}, \sigma_1^{2(j)}, \sigma_2^{2(j)} \sim \text{DiscreteUniform}(\tau)
$
  - $\tau$는 $Y$, $\beta_1^{(j)}$, $\beta_2^{(j)}$, $\sigma_1^2$, $\sigma_2^2$에 의존하여 샘플링됩니다.

#### **7단계**: $j = j+1$로 설정
- $j$를 1 증가시키고, $j \leq n$이면 1단계로 돌아가 반복합니다. $n$은 총 반복 횟수입니다.

### 이 알고리즘의 특징

1. **순차적 샘플링**: 각 파라미터는 다른 파라미터가 고정된 상태에서 샘플링됩니다. 이러한 순차적 갱신 과정은 Gibbs 샘플링의 특징입니다.
   
2. **구조변화 시점 추정**: 가장 중요한 단계는 **구조변화 시점 $\tau$**를 샘플링하는 것입니다. 이는 이전 단계에서 샘플링된 회귀계수와 분산에 의존합니다.

3. **완전 조건부 분포**: 각 단계에서 조건부 분포를 통해 파라미터를 갱신하는 것이 핵심입니다.

### 요약

이 알고리즘은 **구조변화모형**을 추정하기 위해 **깁스 샘플링**을 사용하는 과정입니다. 먼저 초기값을 설정한 후, 회귀 계수와 분산을 샘플링하고, 마지막으로 구조변화 시점을 추정하는 단계로 구성되어 있습니다.


**Gibbs 샘플링**을 통해 **구조변화 시점**을 추정하는 절차를 설명합니다. 책의 수식과 절차를 바탕으로 우리는 구조변화 시점과 그에 따른 회귀 계수 및 분산을 추정할 수 있으며, 이를 통해 데이터가 시간에 따라 어떻게 변화하는지를 분석할 수 있습니다.특히 이 모델은 경제 데이터를 분석하는 데 매우 유용하며, 특히 정책 변화나 외부 충격과 같은 사건이 발생한 시점을 식별하는 데 중요한 도구로 사용됩니다.